<a href="https://colab.research.google.com/github/Mohibul/sockMerchant/blob/master/topic_modelling_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import zipfile
from google.colab import drive

drive.mount('/content/drive/')

# !unzip "/content/drive/My Drive/2020-08-01.zip" -d "/content/drive/My Drive/08-01-2020/"

Mounted at /content/drive/


In [1]:
import pandas as pd
import glob

files = glob.glob("/content/drive/My Drive/08-01-2020/2020-08-01/*.json")
tweet_json = pd.concat([pd.read_json(f) for f in files], ignore_index = True)

tweet_json.head()

,created_at,id_str,full_text,geo,coordinates,retweet_count,favorite_count,lang,u_id_str,u_screen_name,u_location,u_followers_count,u_friends_count,u_favourites_count,u_verified,estimated_loc
0,2020-07-31 23:59:56+00:00,1289350113840660480,RT @slothanova: At least during the 1918 pande...,None,None,140795,0,en,437008571,gasupthewrld,"RAL , NC",181,630,87,False,NC
1,2020-07-31 23:59:56+00:00,1289350113756815360,RT @ScottMonty: Nike's New Ad Is an Editing Ma...,None,None,5,0,en,23242175,Valerie_Hoff,"Atlanta, GA",9096,8947,2804,False,GA
2,2020-07-31 23:59:56+00:00,1289350112108412928,RT @politico: Even the August recess — one of ...,None,None,24,0,en,1154192620949839872,SLejnar,"Massachusetts, USA",199,134,107403,False,MA
3,2020-07-31 23:59:55+00:00,1289350106454532096,RT @kamariholloway: 🚨BREAKING: Another Black w...,None,None,8616,0,en,46279760,SoFaKing_Great_,CT «==» NC,783,922,1714,False,CT
4,2020-07-31 23:59:55+00:00,1289350107960193024,RT @JuliusGoat: Should highways introduce a la...,None,None,370,0,en,3894192132,JesseAChapman1,"Oregon, USA",451,1032,86646,False,OR


In [2]:
tweet_json = tweet_json[tweet_json.lang=='en']
tweet_json.shape

(166988, 16)

In [3]:
import re
import string

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'#','', text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    
    return text

In [24]:
import nltk
nltk.download('punkt')
tweet_json['text_clean'] = tweet_json['full_text'].apply(cleanTxt)
tweet_json['text_clean'] = tweet_json['text_clean'].apply(lambda x: remove_URL(x))
tweet_json['text_clean'] = tweet_json['text_clean'].apply(lambda x: remove_emoji(x))
tweet_json['text_clean'] = tweet_json['text_clean'].apply(lambda x: remove_html(x))
tweet_json['text_clean'] = tweet_json['text_clean'].apply(lambda x: remove_punct(x))

tweet_json['tokenized'] = tweet_json['text_clean'].apply(nltk.word_tokenize)
tweet_json['lower'] = tweet_json['tokenized'].apply(lambda x: [word.lower() for word in x])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
from nltk.corpus import stopwords, wordnet
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop = set(stopwords.words('english'))
tweet_json['stopwords_removed'] = tweet_json['lower'].apply(lambda x: [word for word in x if word not in stop])

tweet_json['pos_tags'] = tweet_json['stopwords_removed'].apply(nltk.tag.pos_tag)
tweet_json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,created_at,id_str,full_text,geo,coordinates,retweet_count,favorite_count,lang,u_id_str,u_screen_name,u_location,u_followers_count,u_friends_count,u_favourites_count,u_verified,estimated_loc,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,lemma_str
0,2020-07-31 23:59:56+00:00,1289350113840660480,RT @slothanova: At least during the 1918 pande...,None,None,140795,0,en,437008571,gasupthewrld,"RAL , NC",181,630,87,False,NC,At least during the 1918 pandemic they had co...,"[At, least, during, the, 1918, pandemic, they,...","[at, least, during, the, 1918, pandemic, they,...","[least, 1918, pandemic, cocaine, soda]","[(least, JJS), (1918, CD), (pandemic, JJ), (co...","[(rt, n), (slothanova, n), (least, a), (1918, ...","[rt, slothanova, least, 1918, pandemic, cocain...",rt slothanova least 1918 pandemic cocaine soda
1,2020-07-31 23:59:56+00:00,1289350113756815360,RT @ScottMonty: Nike's New Ad Is an Editing Ma...,None,None,5,0,en,23242175,Valerie_Hoff,"Atlanta, GA",9096,8947,2804,False,GA,Nikes New Ad Is an Editing Marvel and Maybe t...,"[Nikes, New, Ad, Is, an, Editing, Marvel, and,...","[nikes, new, ad, is, an, editing, marvel, and,...","[nikes, new, ad, editing, marvel, maybe, best,...","[(nikes, NNS), (new, JJ), (ad, NN), (editing, ...","[(rt, a), (scottmonty, n), (nikes, n), (new, a...","[rt, scottmonty, nike, new, ad, edit, marvel, ...",rt scottmonty nike new ad edit marvel maybe be...
2,2020-07-31 23:59:56+00:00,1289350112108412928,RT @politico: Even the August recess — one of ...,None,None,24,0,en,1154192620949839872,SLejnar,"Massachusetts, USA",199,134,107403,False,MA,Even the August recess — one of the Congress’...,"[Even, the, August, recess, —, one, of, the, C...","[even, the, august, recess, —, one, of, the, c...","[even, august, recess, —, one, congress, ’, ha...","[(even, RB), (august, JJ), (recess, NN), (—, I...","[(rt, n), (politico, n), (even, r), (august, a...","[rt, politico, even, august, recess, —, one, c...",rt politico even august recess — one congress ...
3,2020-07-31 23:59:55+00:00,1289350106454532096,RT @kamariholloway: 🚨BREAKING: Another Black w...,None,None,8616,0,en,46279760,SoFaKing_Great_,CT «==» NC,783,922,1714,False,CT,BREAKING Another Black woman bought a Tesla i...,"[BREAKING, Another, Black, woman, bought, a, T...","[breaking, another, black, woman, bought, a, t...","[breaking, another, black, woman, bought, tesl...","[(breaking, VBG), (another, DT), (black, JJ), ...","[(rt, v), (kamariholloway, n), (breaking, v), ...","[rt, kamariholloway, break, another, black, wo...",rt kamariholloway break another black woman bu...
4,2020-07-31 23:59:55+00:00,1289350107960193024,RT @JuliusGoat: Should highways introduce a la...,None,None,370,0,en,3894192132,JesseAChapman1,"Oregon, USA",451,1032,86646,False,OR,Should highways introduce a lane for drunk dr...,"[Should, highways, introduce, a, lane, for, dr...","[should, highways, introduce, a, lane, for, dr...","[highways, introduce, lane, drunk, drivers, su...","[(highways, NNS), (introduce, VBP), (lane, JJ)...","[(rt, n), (juliusgoat, n), (highways, v), (int...","[rt, juliusgoat, highways, introduce, lane, dr...",rt juliusgoat highways introduce lane drunk dr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185988,2020-08-01 03:59:54+00:00,1289410501340880896,"RT @ethanisimon: Meanwhile, in an alternate ti...",None,None,716,0,en,2887560107,michelemg5,"Tennessee, USA",80,1494,65106,False,TN,Meanwhile in an alternate timeline we’re 3 mo...,"[Meanwhile, in, an, alternate, timeline, we, ’...","[meanwhile, in, an, alternate, timeline, we, ’...","[meanwhile, alternate, timeline, ’, 3, months,...","[(meanwhile, RB), (alternate, JJ), (timeline, ...","[(rt, n), (ethanisimon, n), (meanwhile, r), (a...","[rt, ethanisimon, meanwhile, alternate, timeli...",rt ethanisimon meanwhile alternate timeline ’ ...
185989,2020-08-01 03:59:54+00:00,1289410503878443008,RT @slothanova: At least during the 

In [26]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


tweet_json['wordnet_pos'] = tweet_json['pos_tags'].apply(
    lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

tweet_json.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,created_at,id_str,full_text,geo,coordinates,retweet_count,favorite_count,lang,u_id_str,u_screen_name,u_location,u_followers_count,u_friends_count,u_favourites_count,u_verified,estimated_loc,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,lemma_str
0,2020-07-31 23:59:56+00:00,1289350113840660480,RT @slothanova: At least during the 1918 pande...,None,None,140795,0,en,437008571,gasupthewrld,"RAL , NC",181,630,87,False,NC,At least during the 1918 pandemic they had co...,"[At, least, during, the, 1918, pandemic, they,...","[at, least, during, the, 1918, pandemic, they,...","[least, 1918, pandemic, cocaine, soda]","[(least, JJS), (1918, CD), (pandemic, JJ), (co...","[(least, a), (1918, n), (pandemic, a), (cocain...","[rt, slothanova, least, 1918, pandemic, cocain...",rt slothanova least 1918 pandemic cocaine soda
1,2020-07-31 23:59:56+00:00,1289350113756815360,RT @ScottMonty: Nike's New Ad Is an Editing Ma...,None,None,5,0,en,23242175,Valerie_Hoff,"Atlanta, GA",9096,8947,2804,False,GA,Nikes New Ad Is an Editing Marvel and Maybe t...,"[Nikes, New, Ad, Is, an, Editing, Marvel, and,...","[nikes, new, ad, is, an, editing, marvel, and,...","[nikes, new, ad, editing, marvel, maybe, best,...","[(nikes, NNS), (new, JJ), (ad, NN), (editing, ...","[(nikes, n), (new, a), (ad, n), (editing, v), ...","[rt, scottmonty, nike, new, ad, edit, marvel, ...",rt scottmonty nike new ad edit marvel maybe be...
2,2020-07-31 23:59:56+00:00,1289350112108412928,RT @politico: Even the August recess — one of ...,None,None,24,0,en,1154192620949839872,SLejnar,"Massachusetts, USA",199,134,107403,False,MA,Even the August recess — one of the Congress’...,"[Even, the, August, recess, —, one, of, the, C...","[even, the, august, recess, —, one, of, the, c...","[even, august, recess, —, one, congress, ’, ha...","[(even, RB), (august, JJ), (recess, NN), (—, I...","[(even, r), (august, a), (recess, n), (—, n), ...","[rt, politico, even, august, recess, —, one, c...",rt politico even august recess — one congress ...
3,2020-07-31 23:59:55+00:00,1289350106454532096,RT @kamariholloway: 🚨BREAKING: Another Black w...,None,None,8616,0,en,46279760,SoFaKing_Great_,CT «==» NC,783,922,1714,False,CT,BREAKING Another Black woman bought a Tesla i...,"[BREAKING, Another, Black, woman, bought, a, T...","[breaking, another, black, woman, bought, a, t...","[breaking, another, black, woman, bought, tesl...","[(breaking, VBG), (another, DT), (black, JJ), ...","[(breaking, v), (another, n), (black, a), (wom...","[rt, kamariholloway, break, another, black, wo...",rt kamariholloway break another black woman bu...
4,2020-07-31 23:59:55+00:00,1289350107960193024,RT @JuliusGoat: Should highways introduce a la...,None,None,370,0,en,3894192132,JesseAChapman1,"Oregon, USA",451,1032,86646,False,OR,Should highways introduce a lane for drunk dr...,"[Should, highways, introduce, a, lane, for, dr...","[should, highways, introduce, a, lane, for, dr...","[highways, introduce, lane, drunk, drivers, su...","[(highways, NNS), (introduce, VBP), (lane, JJ)...","[(highways, n), (introduce, v), (lane, a), (dr...","[rt, juliusgoat, highways, introduce, lane, dr...",rt juliusgoat highways introduce lane drunk dr...


In [27]:
wnl = WordNetLemmatizer()

tweet_json['lemmatized'] = tweet_json['wordnet_pos'].apply(
    lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

tweet_json['lemmatized'] = tweet_json['lemmatized'].apply(
    lambda x: [word for word in x if word not in stop])

tweet_json['lemma_str'] = [' '.join(map(str, l)) for l in tweet_json['lemmatized']]

tweet_json.head()

,created_at,id_str,full_text,geo,coordinates,retweet_count,favorite_count,lang,u_id_str,u_screen_name,u_location,u_followers_count,u_friends_count,u_favourites_count,u_verified,estimated_loc,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,lemma_str
0,2020-07-31 23:59:56+00:00,1289350113840660480,RT @slothanova: At least during the 1918 pande...,None,None,140795,0,en,437008571,gasupthewrld,"RAL , NC",181,630,87,False,NC,At least during the 1918 pandemic they had co...,"[At, least, during, the, 1918, pandemic, they,...","[at, least, during, the, 1918, pandemic, they,...","[least, 1918, pandemic, cocaine, soda]","[(least, JJS), (1918, CD), (pandemic, JJ), (co...","[(least, a), (1918, n), (pandemic, a), (cocain...","[least, 1918, pandemic, cocaine, soda]",least 1918 pandemic cocaine soda
1,2020-07-31 23:59:56+00:00,1289350113756815360,RT @ScottMonty: Nike's New Ad Is an Editing Ma...,None,None,5,0,en,23242175,Valerie_Hoff,"Atlanta, GA",9096,8947,2804,False,GA,Nikes New Ad Is an Editing Marvel and Maybe t...,"[Nikes, New, Ad, Is, an, Editing, Marvel, and,...","[nikes, new, ad, is, an, editing, marvel, and,...","[nikes, new, ad, editing, marvel, maybe, best,...","[(nikes, NNS), (new, JJ), (ad, NN), (editing, ...","[(nikes, n), (new, a), (ad, n), (editing, v), ...","[nike, new, ad, edit, marvel, maybe, best, cov...",nike new ad edit marvel maybe best covid spot yet
2,2020-07-31 23:59:56+00:00,1289350112108412928,RT @politico: Even the August recess — one of ...,None,None,24,0,en,1154192620949839872,SLejnar,"Massachusetts, USA",199,134,107403,False,MA,Even the August recess — one of the Congress’...,"[Even, the, August, recess, —, one, of, the, C...","[even, the, august, recess, —, one, of, the, c...","[even, august, recess, —, one, congress, ’, ha...","[(even, RB), (august, JJ), (recess, NN), (—, I...","[(even, r), (august, a), (recess, n), (—, n), ...","[even, august, recess, —, one, congress, ’, ha...",even august recess — one congress ’ hallow tra...
3,2020-07-31 23:59:55+00:00,1289350106454532096,RT @kamariholloway: 🚨BREAKING: Another Black w...,None,None,8616,0,en,46279760,SoFaKing_Great_,CT «==» NC,783,922,1714,False,CT,BREAKING Another Black woman bought a Tesla i...,"[BREAKING, Another, Black, woman, bought, a, T...","[breaking, another, black, woman, bought, a, t...","[breaking, another, black, woman, bought, tesl...","[(breaking, VBG), (another, DT), (black, JJ), ...","[(breaking, v), (another, n), (black, a), (wom...","[break, another, black, woman, buy, tesla, mid...",break another black woman buy tesla middle pan...
4,2020-07-31 23:59:55+00:00,1289350107960193024,RT @JuliusGoat: Should highways introduce a la...,None,None,370,0,en,3894192132,JesseAChapman1,"Oregon, USA",451,1032,86646,False,OR,Should highways introduce a lane for drunk dr...,"[Should, highways, introduce, a, lane, for, dr...","[should, highways, introduce, a, lane, for, dr...","[highways, introduce, lane, drunk, drivers, su...","[(highways, NNS), (introduce, VBP), (lane, JJ)...","[(highways, n), (introduce, v), (lane, a), (dr...","[highway, introduce, lane, drunk, driver, sugg...",highway introduce lane drunk driver suggestion...


In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=25, max_features=2000, use_idf=True)
tfidf = tfidf_vectorizer.fit_transform(tweet_json.lemma_str)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
doc_term_matrix_tfidf = pd.DataFrame(tfidf.toarray(), columns=list(tfidf_feature_names))

doc_term_matrix_tfidf

,10,100,1000,10000,11,12,1214,13,14,15,150000,150k,153000,156747,15m,16,17,175000,18,182000,19,1918,1st,20,200,2016,2019,2020,202021,2021,21,22,23,24,25,254k,257,260,27,2nd,...,woman,wonder,wont,word,work,worker,world,worldwide,worn,worried,worry,worst,worth,would,wouldnt,wow,write,wrong,wtf,wuhan,ya,yale,yall,yeah,year,yes,yesterday,yet,ymca,yo,york,you,young,youre,youth,youve,zero,zinc,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.492697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.358188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.492697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [30]:
nmf = NMF(n_components=2, random_state=0, alpha=.1, init='nndsvd').fit(tfidf)
no_top_words = 10
for topic_idx, topic in enumerate(nmf.components_):
    print('Topic %d:' % (topic_idx+1))
    print(' '.join([tfidf_feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

Topic 1:
pandemic trump want let one american administration dead use mince
Topic 2:
thread restaurant eat host covid19 case first state would test


In [31]:
from gensim import matutils, models
import scipy.sparse

In [32]:
tdm = doc_term_matrix_tfidf.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,166948,166949,166950,166951,166952,166953,166954,166955,166956,166957,166958,166959,166960,166961,166962,166963,166964,166965,166966,166967,166968,166969,166970,166971,166972,166973,166974,166975,166976,166977,166978,166979,166980,166981,166982,166983,166984,166985,166986,166987
10,0.0,0.0,0.0,0.0,0.0,0.214757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)